In [1]:
%load_ext autoreload
%autoreload 2

from pricing.operations.functions import *
from pricing.operations.utils import *
from pprint import pprint
import os


from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

from pricing.operations.experiments import level_1_kind_1_courses_1

# Testing databricks connection

In [4]:
spark.sql("select * from querobolsa_production.offers limit 10").toPandas()

,id,course_id,university_offer_id,seats_balance_id,limited,enabled,visible,position,offered_price,discount_percentage,...,real_discount,commercial_discount,commercial_price,real_total_absolute_discount,boleto_due_date,billing_integration,expected_revenue,regressive_commercial_discount,show_on_main_search,status
0,48745135,1880593,8560846,21285804,True,False,True,1.0,119.01,56.39,...,56.39,45.000000000000000000,150.090000000000000000,None,NaN,False,272.890000000000000000,0E-18,True,disabled
1,48745136,5166970,8557316,21452219,True,False,False,2.0,162.97,52.92,...,52.92,44.040000000000000000,169.610000000000000000,None,NaN,False,162.970000000000000000,0E-18,True,disabled
2,48745137,1880621,8560894,21285806,True,False,True,1.0,119.01,56.39,...,56.39,45.000000000000000000,150.090000000000000000,None,NaN,False,272.890000000000000000,0E-18,True,disabled
3,48745138,3938710,8613735,21452244,True,False,False,2.0,162.97,50.29,...,50.29,46.610000000000000000,170.480000000000000000,None,NaN,False,162.970000000000000000,0E-18,True,disabled
4,48745139,3941000,8561358,21285808,True,False,True,1.0,135.27,50.43,...,50.43,45.000000000000000000,150.090000000000000000,None,NaN,False,272.890000000000000000,0E-18,True,disabled
5,48745140,3938728,8554902,21452245,True,False,False,2.0,160.88,55.86,...,55.86,54.000000000000000000,167.660000000000000000,None,NaN,False,160.880000000000000000,0E-18,True,disabled
6,48745141,5033073,8561795,21285811,True,False,True,1.0,126.65,53.59,...,53.59,45.000000000000000000,150.090000000000000000,None,NaN,False,272.890000000000000000,0E-18,True,disabled
7,48745142,4503124,8404873,21452246,True,False,False,2.0,217.53,52.30,...,52.30,42.320000000000000000,228.020000000000000000,None,NaN,False,217.530000000000000000,0E-18,True,disabled
8,48745143,3938806,8559743,21452248,True,False,False,2.0,160.88,55.86,...,55.86,54.000000000000000000,167.660000000000000000,None,NaN,False,160.880000000000000000,0E-18,True,disabled
9,48745144,1884970,8561859,21285814,True,False,True,NaN,141.55,48.13,...,48.13,45.000000000000000000,150.090000000000000000,None,NaN,False,149.900000000000000000,0E-18,True,disabled


# Updating Experiments

In [2]:
check_experiments(spark=spark,update=True)  # verify all without update

Using as base_host:  https://querobolsa.com.br/
OK
level 1 kind 1 courses 1 ID: 56
target_offers_sql OK

Checking alternative: seasonality plus  | id:  321  ...
     fee_formula_sql is outdated ...
[ 2020-02-07 17:19:32 ] Updating fee_formula_sql ...

Checking alternative: seasonality minus  | id:  322  ...
     fee_formula_sql is outdated ...
[ 2020-02-07 17:19:32 ] Updating fee_formula_sql ...

Checking alternative: take rate  | id:  366  ...
Nothing to change

Checking alternative: seasonality lowest  | id:  374  ...
     fee_formula_sql is outdated ...
[ 2020-02-07 17:19:32 ] Updating fee_formula_sql ...

Checking alternative: seasonality lower  | id:  373  ...
     fee_formula_sql is outdated ...
[ 2020-02-07 17:19:33 ] Updating fee_formula_sql ...

Checking alternative: seasonality minus 20  | id:  383  ...
     fee_formula_sql is outdated ...
[ 2020-02-07 17:19:33 ] Updating fee_formula_sql ...

Checking alternative: dynamic pricing  | id:  320  ...
     fee_formula_sql is outda

In [5]:
check_experiment_difference(57)  # verify single experiment by id

Using as base_host:  https://querobolsa.com.br/

Experiment being checked:  level 1 kind 1 courses 2

Checking alternative: seasonality minus  | id:  327  ...
Nothing to change

Checking alternative: dynamic pricing  | id:  325  ...
Nothing to change

Checking alternative: seasonality lower  | id:  375  ...
Nothing to change

Checking alternative: seasonality lowest  | id:  376  ...
Nothing to change

Checking alternative: seasonality plus  | id:  326  ...
Nothing to change

Checking alternative: seasonality minus 20  | id:  384  ...
Nothing to change

Checking alternative: baseline  | id:  324  ...
Nothing to change

Checking alternative: take rate  | id:  367  ...
Nothing to change


In [6]:
update_experiment(57)

Using as base_host:  https://querobolsa.com.br/

Experiment being updated:  level 1 kind 1 courses 2

Checking alternative: seasonality minus  | id:  327  ...
Nothing to change

Checking alternative: dynamic pricing  | id:  325  ...
Nothing to change

Checking alternative: seasonality lower  | id:  375  ...
Nothing to change

Checking alternative: seasonality lowest  | id:  376  ...
Nothing to change

Checking alternative: seasonality plus  | id:  326  ...
Nothing to change

Checking alternative: seasonality minus 20  | id:  384  ...
Nothing to change

Checking alternative: baseline  | id:  324  ...
Nothing to change

Checking alternative: take rate  | id:  367  ...
Nothing to change

Finished updating experiment


## Recalculate formulas

In [10]:
#### USE JUST WITH FILTERS!!! ###

experiment_id = 56

recalculate_alternatives(experiment_id, 'Recalculate reason', course_ids = [10])


Recalculating using as base_host:  http://172.17.0.1:3000/   -> with filters:  {'course_id': [10]}
Are you sure [y,N]? y
[ 2020-02-06 19:32:40 ] Sucess.
Check job at https://app.datadoghq.com/logs?cols=host%2Cservice%2C%40timestamp&event&from_ts=1576593859294&index=main&live=true&messageDisplay=inline&query=%22Starting+experiment+setup+job%22+OR++%22Recalculating+alternative+fees+for+experiment%22+OR+%22Recalculating+alternative+fees+for+experiment%22+OR+%22End+of+recalculations+for%22+OR+%22Experiment+setup+job+finished%22&stream_sort=asc&to_ts=1576597459294


<Response [200]>